# 🎙️ Generate TTS & Push to Git

Tạo audio cho `<teacher_script>` và push lên GitHub.

In [ ]:
# @title 1️⃣ Cấu hình
GIT_TOKEN = "" # @param {type:"string"}
GIT_REPO = "nmnhut-it/english-learning-app" # @param {type:"string"}
GIT_BRANCH = "main" # @param {type:"string"}
VOICE = "vi-VN-HoaiMyNeural" # @param ["vi-VN-HoaiMyNeural", "vi-VN-NamMinhNeural"]

# Install
!pip install edge-tts -q
print("✅ Ready")

In [ ]:
# @title 2️⃣ Clone repo
import os

!rm -rf repo
!git clone https://{GIT_TOKEN}@github.com/{GIT_REPO}.git repo --branch {GIT_BRANCH} --depth 1
os.chdir('repo')

# List markdown files
md_files = !find v2/data/voice-lectures -name "*.md" 2>/dev/null
print(f"\n📁 Tìm thấy {len(md_files)} file markdown:")
for f in md_files[:10]:
    print(f"  {f}")
if len(md_files) > 10:
    print(f"  ... và {len(md_files)-10} file nữa")

In [ ]:
# @title 3️⃣ Chọn file để gen audio
MD_FILE = "v2/data/voice-lectures/g6/unit-07/getting-started.md" # @param {type:"string"}

import re
import hashlib

with open(MD_FILE, 'r', encoding='utf-8') as f:
    content = f.read()

pattern = r'<teacher_script([^>]*)>([\s\S]*?)</teacher_script>'
matches = re.findall(pattern, content, re.IGNORECASE)

scripts = []
for i, (attrs, text) in enumerate(matches):
    text = text.strip()
    if text and 'href=' not in attrs:  # Skip if already has audio
        h = hashlib.md5(text.encode()).hexdigest()[:8]
        scripts.append({'i': i, 'text': text, 'hash': h, 'file': f'ts_{i:03d}_{h}.mp3', 'attrs': attrs})

print(f"📝 {len(scripts)} scripts cần gen audio (bỏ qua {len(matches)-len(scripts)} đã có)")
for s in scripts[:5]:
    print(f"  {s['i']+1}. {s['text'][:50]}...")

In [ ]:
# @title 4️⃣ Generate audio
import edge_tts
import asyncio

audio_dir = os.path.join(os.path.dirname(MD_FILE), 'audio')
os.makedirs(audio_dir, exist_ok=True)

async def gen(text, path):
    c = edge_tts.Communicate(text, VOICE)
    await c.save(path)

async def gen_all():
    for s in scripts:
        path = os.path.join(audio_dir, s['file'])
        print(f"🔊 [{s['i']+1}/{len(scripts)}] {s['file']}")
        try:
            await gen(s['text'], path)
            s['ok'] = True
        except Exception as e:
            print(f"   ❌ {e}")
            s['ok'] = False

await gen_all()
print(f"\n✅ Done! Generated {sum(1 for s in scripts if s.get('ok'))} files")

In [ ]:
# @title 5️⃣ Update markdown với href
updated = content

for s in scripts:
    if not s.get('ok'):
        continue
    old = f'<teacher_script{s["attrs"]}>{s["text"]}</teacher_script>'
    href = f'href="audio/{s["file"]}"'
    new_attrs = f'{s["attrs"]} {href}'.strip()
    new = f'<teacher_script {new_attrs}>{s["text"]}</teacher_script>'
    updated = updated.replace(old, new)

with open(MD_FILE, 'w', encoding='utf-8') as f:
    f.write(updated)

print(f"✅ Updated {MD_FILE}")

In [ ]:
# @title 6️⃣ Push to Git
!git config user.email "colab@example.com"
!git config user.name "Colab TTS Bot"
!git add .
!git status
!git commit -m "Add TTS audio for {os.path.basename(MD_FILE)}"
!git push
print("\n✅ Pushed to Git!")

---
## 🔄 Batch: Gen cho nhiều file
Chạy cell dưới để gen audio cho TẤT CẢ file markdown.

In [ ]:
# @title 🔄 Batch generate all
import edge_tts, asyncio, re, hashlib, os

async def process_file(md_path):
    with open(md_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    matches = re.findall(r'<teacher_script([^>]*)>([\s\S]*?)</teacher_script>', content, re.I)
    scripts = []
    for i, (attrs, text) in enumerate(matches):
        text = text.strip()
        if text and 'href=' not in attrs:
            h = hashlib.md5(text.encode()).hexdigest()[:8]
            scripts.append({'i': i, 'text': text, 'hash': h, 'file': f'ts_{i:03d}_{h}.mp3', 'attrs': attrs})
    
    if not scripts:
        return 0
    
    audio_dir = os.path.join(os.path.dirname(md_path), 'audio')
    os.makedirs(audio_dir, exist_ok=True)
    
    for s in scripts:
        path = os.path.join(audio_dir, s['file'])
        try:
            c = edge_tts.Communicate(s['text'], VOICE)
            await c.save(path)
            s['ok'] = True
        except:
            s['ok'] = False
    
    # Update markdown
    updated = content
    for s in scripts:
        if not s.get('ok'): continue
        old = f'<teacher_script{s["attrs"]}>{s["text"]}</teacher_script>'
        new = f'<teacher_script {s["attrs"]} href="audio/{s["file"]}">{s["text"]}</teacher_script>'
        updated = updated.replace(old, new)
    
    with open(md_path, 'w', encoding='utf-8') as f:
        f.write(updated)
    
    return sum(1 for s in scripts if s.get('ok'))

# Process all
total = 0
for md in md_files:
    print(f"📄 {md}")
    n = await process_file(md)
    total += n
    if n: print(f"   ✅ {n} audio files")

print(f"\n🎉 Total: {total} audio files generated")

# Push
!git add . && git commit -m "Add TTS audio for all voice lectures" && git push
print("✅ Pushed!")